<div class="alert alert-block alert-danger">

# FIT5196 Task 2 in Assessment 1
    
#### Student Name: Tsz Yan CHUNG
#### Student ID: 32973381

Date: 24/8/2022

Environment: Google colab

Libraries used:
* os (for interacting with the operating system, mainly used to access directory paths) 
* pandas 1.1.0 (for dataframe, installed and imported) 
* multiprocessing (for performing processes on multi cores, included in Python 3.6.9 package) 
* itertools (for performing operations on iterables)
* nltk 3.5 (Natural Language Toolkit, installed and imported)
* nltk.collocations (for finding bigrams, installed and imported)
* nltk.tokenize (for tokenization, installed and imported)
* nltk.stem (for stemming the tokens, installed and imported)
* math (for getting threshold value)
* PyPDF2 (for reading pdf file and storing values of pdf file into python variables)
* sklearn.feature_extraction (for creating sparse matrix for vectorized tokens)

    </div>

<div class="alert alert-block alert-info">
    
## Table of Contents

</div>

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Input File](#examine) <br>
[4. Parsing Data](#parse) <br>
$\;\;\;\;$[4.1. Tokenization - Unigram](#tokenize) <br>
$\;\;\;\;$[4.2. Finding First 200 Meaningful Bigrams](#bigrams) <br>
$\;\;\;\;$[4.3. Re-tokenization](#retoken) <br>
$\;\;\;\;$[4.4. Remove Context-Independent Stopwords and words with length less than 3](#rm1) <br>
$\;\;\;\;$[4.5. Remove Context-Dependent Stopwords and Rarewords](#rm2) <br>
$\;\;\;\;$[4.6. Tokens Stemming](#token_stem) <br>
$\;\;\;\;$[4.7. Token Vectorization](#token_vec) <br>
[5. Writing Output Files](#rm2) <br>
$\;\;\;\;$[5.1. Vocabulary List](#output2) <br>
$\;\;\;\;$[5.2. Sparse Matrix](#output1) <br>
[6. References](#Ref) <br>

<div class="alert alert-block alert-success">
    
## 1.  Introduction  <a class="anchor" name="Intro"></a>

This assessment concerns textual data and the aim is to extract data, process them, and transform them into a proper format. The dataset provided is in the format of a PDF file containing pages of information. The task is to correctly extract data from the file, so that follow up operations could be performed to achieve the aim of this assessment efficiently.
Several data processing methods will be performed in this assignment, including tokenization, words removal, stemming and vectorization.
In this end of this task, 2 output file will be created and will be submitted without further editting.

<div class="alert alert-block alert-success">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>

In this assessment, any python packages is permitted to be used. The following packages were used to accomplish the related tasks:

* **os:** to interact with the operating system, e.g. navigate through folders to read files
* **re:** to define and use regular expressions
* **pandas:** to work with dataframes
* **multiprocessing:** to perform processes on multi cores for fast performance 
* **nltk.probability:** to create frequency distribution for tokens among PID
* **nltk.stem:** to stem tokens and prevent words with duplicated meanings
* **nltk.collocation:** to find bigrams that fulfill the requirements
* **nltk.tokenize.RegexpTokenizer:** to find all unigrams from corpus
* **nltk.tokenize.MWETokenizer:** to merge meaningful collocation of unigrams (bigrams) from list of unigrams
* **sklearn.feature_extraction.text:** to extract frequency distribution array and items' name from a dictionary, in order to create the final sparse matrix output
* **itertools.chain:** to combine values of a dictionary across dictioanry's keys


In [1]:
import os
import re
!pip install PyPDF2
import PyPDF2
# import langid
import pandas as pd
import multiprocessing
from itertools import chain
import nltk
from nltk.probability import *
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
import math


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


-------------------------------------

<div class="alert alert-block alert-success">
    
## 3.  Examining Input File and Data Extraction<a class="anchor" name="examine"></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


To examine the content of the PDF file, "PyPDF2" library is used to process the PDF file, and a variable is created to store all values from the PDF file.


In [3]:
os.chdir("/content/drive/Shareddrives/FIT5196-s2-2022/A1/Task2/input_data")
file = open("32973381_task2.pdf", "rb")
file_reader = PyPDF2.PdfFileReader(file)

# Go through all pages in PDF file
pages = ''
n = 0
while n < file_reader.numPages:
  pages += file_reader.getPage(n).extractText()
  n += 1

From reading the PDF file and build up basic understanding on the file, regular expression methods could be set up to find all product IDs and reviews regarding the specific product ID


In [4]:
# Find product ID
id_list = re.findall(r"(?:([\w\d]+)\n\[(?:'|\"))|(?:'|\")\]\n([\w\d]+)", pages)
PID = []

# As there are two capturing groups in my regular expression method for PID,
# the results are looped through and remove unwanted (empty) data 
for sub_list in id_list:
  for item in sub_list:
    if len(item) > 0:
      PID.append(item)

# Get reviews
reviews = re.findall(r"(?<=\[)(?:'|\")[\s\S\w\W\d\D]*?(?:'|\")(?=\])", pages)

To examine the extracted information, a quick check on length of PID and reviews list is conducted.

As for each PID, a list is containing all the passages of reviews for that product. Therefore PID and reviews lists should have the same length if the RE methods are performed correctly

In [5]:
# Quick check on length of PID and reviews list

len(PID) == len(reviews)

True

In [6]:
# Create dictionary combining PID and the corresponding reviews information, 
# With PID as the keys of the dictionary, and reviews as the values
data_dict = {}
for key, value in zip(PID, reviews):
  data_dict[key] = value

<div class="alert alert-block alert-success">
    
## 4.  Parsing Data<a class="anchor" name="parse"></a>

In this session, data read from the file will be processed, such that there is a recognizable patterns to be found in each individual reviews passage for every Product ID. 
Tasks performed are as follows: <br>
$\;\;\;\;$[4.1. Tokenization - Unigram](#tokenize) <br>
$\;\;\;\;$[4.2. Finding First 200 Meaningful Bigrams](#bigram) <br>
$\;\;\;\;$[4.3. Re-tokenization](#retoken) <br>
$\;\;\;\;$[4.4. Remove Context-Independent Stopwords and words with length less than 3](#rm1) <br>
$\;\;\;\;$[4.5. Remove Context-Dependent Stopwords and Rarewords](#rm2) <br>
$\;\;\;\;$[4.6. Tokens Stemming](#token_stem) <br>
$\;\;\;\;$[4.7. Token Vectorization](#token_vec) <br>

<div class="alert alert-block alert-warning">
    
### 4.1. Tokenization - Unigram<a class="anchor" name="tokenize"></a>

Now that a dictionary connecting a product ID to its corresponding product review is set up, tokenization methods are set up to break down words in review column into a list of separate tokens using the NLTK library.


In [7]:
# Tokenize data and extract unigrams from data
tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")

unigram_dict = {}
for n in data_dict.keys():
  tokens = data_dict[n].lower()
  unigram_dict[n] = tokenizer.tokenize(tokens)

<div class="alert alert-block alert-warning">
    
### 4.2. Tokenization - First 200 meaningful Bigrams<a class="anchor" name="bigram"></a>

After getting all unigrams. I proceed to generate the bigram collocations.

The chain.from_iterable function here combining a word to another.
The returned list by this function will give a list of every word from all reviews in the data, separated by commas.




In [8]:
all_words = list(chain.from_iterable(unigram_dict.values()))

Apply the following functions from nltk.collocation library to generate the first 200 meaningful bigrams:
* BigramAssocMeasures()
* BigramCollocationFinder.from_words()
* apply_freq_filter(20)
* apply_word_filter(lambda w: len(w) < 3)
* nbest(bigram_measures.pmi, 100)

In this task, the rule of not allowing any token with a length less than 3 is also applied

The PMI measure is used to get the first 200 meaningful bigrams



In [9]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(all_words) # look for bigrams from "all_words"
bigram_finder.apply_freq_filter(20) # filter for bigrams appeared for more than 20 times in corpus
bigram_finder.apply_word_filter(lambda w: len(w) < 3) 
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top 200 bigrams by PMI measure

<div class="alert alert-block alert-warning">
    
### 4.3. Tokenization - Re-tokenize review data<a class="anchor" name="retoken"></a>

The previous task conducted in 4.1 tokenized review data with unigrams.
However, after getting the top 200 bigrams from the document, the bigrams should also be introduced to ensure words with special meanings are not left out.

A function that assist on this task is ***MWETokenizer***, which referenced the established top_200_bigrams and check for meaningful collocation phrases from unigram_dict defined in task 4.1

In [10]:
mwetokenizer = MWETokenizer(top_200_bigrams)

mwe_tokens = {}
for pid, review in unigram_dict.items():
    mwe_tokens[pid] = mwetokenizer.tokenize(review)

<div class="alert alert-block alert-warning">
    
### 4.4. Remove Words: Context-Independent Stopwords, Words with length less than 3<a class="anchor" name="rm1"></a>
To remove words/ phrases that contain only little value, first extract all these context-independent stopwords listed on the given "stopwords_en.txt" file.

In [11]:
# Extract all stopwords
with open ("stopwords_en.txt", 'r', encoding = "utf-8") as f:
  stopwords = f.read().split('\n')

# Remove duplicate stopwords values in case there is any
stopwords = list(set(stopwords))

In the previous process where top 200 bigrams were extracted, words with length less than 3 were filtered out. 
However, this has not be done on the list of unigrams.
So that in this step these words will be removed from the defined "mwe_tokens" variable.

In [12]:
mwe_dict = {}
for key, value in mwe_tokens.items():
  token_list = []
  for token in value:
    # apply filter where length of tokens is greater than 3, and it is not on the stopwords list
    if len(token) >= 3 and token not in stopwords:
      token_list.append(token)
    mwe_dict[key] = token_list

At this stage, we have a dictionary of tokenized words, whose keys are indicative of PID of reviews. The list of tokens also include the top 200 meaningful bigrams achieved using PMI measure.


<div class="alert alert-block alert-warning">
    
### 4.5. Remove Words: Context-Dependent Stopwords, Rare tokens<a class="anchor" name="rm2"></a>
Next, context-depedent stopwords are to be removed. These words are not necessariliy typical stopwords in English, but they are uninformative in a way that they are too common and provide little information distinguishing one review to another. The threshold value is given by "ceil(Number_of_PIDs/ 2)"

In [13]:
# Create a dictionary for Product ID, and the corresponding tokens contained in its reviews
# However, each token only exist once for each Product ID (i.e. no duplicate values inside each key (PID))
unique_dict = {}
for key, value in mwe_dict.items():
  tokens = []
  for word in set(value):
    tokens.append(word)
  unique_dict[key] = tokens

# Create a list containing all tokens across all reviews for differnt PID
all_words_tokens = list(chain.from_iterable(unique_dict.values()))

# Calculating the frequency of a token appearing across different PID
# In this case, max is 100, min is 0, as each token list for a PID does not have duplicate value
frequency = FreqDist(all_words_tokens)

# As instructed, the top 200 bigams must be included in our final output,
# These phrases are values needed to be kept
bigram_phrases_200 = ["_".join(bigram) for bigram in top_200_bigrams]

# Create threshold level
threshold = math.ceil(len(mwe_dict)/2)

In [14]:
# define context-dependent stopwords
context_dependent = []
for token, freq in frequency.items():
  if freq >= threshold and token not in bigram_phrases_200:
    context_dependent.append(token)

Words that appear in less than 10 PID's reviews are also considered as stopwords, which have to be removed.

In [15]:
# define rare tokens
rare_tokens = []
for token, freq in frequency.items():
  if freq <10:
    rare_tokens.append(token)

Removing context_dependent stopwords and rare tokens from "all_words_tokens" list which contains all tokens from every reviews.
Those will be the vocabulary required for further operations.

In [16]:
vocab = []
for token in all_words_tokens:
  if token not in context_dependent and token not in rare_tokens:
    vocab.append(token)

# Remove dupliates 
vocab = list(set(vocab))
len(vocab)

1219

<div class="alert alert-block alert-warning">
    
### 4.6. Tokens Stemming<a class="anchor" name="token_stem"></a>
For different words in English, they could be representing the same meaning due to the language grammar. Stemming method is used here to remove the grammar limitation.
So that words with slightly different spelling or formation will be classified as a same word by removing some uses of grammar. This process will be done by the function "Porter stemmer".

In [17]:
# Apply stemming method on "vocab" list
stemmer = PorterStemmer()
vocab_stemmed = [stemmer.stem(i) for i in vocab]

# Remove duplicate values after stemming, tokens with similar meaning will be grouped
vocab_stemmed = sorted(list(set(vocab_stemmed)))

# Given that some tokens might resulted in length smaller than 3 after stemming
# To strictly follow the requirements of tokens, those have to removed as well.
for word in vocab_stemmed:
  if len(word) < 3:
    vocab_stemmed.remove(word)


After assigning index to vocab_stemmed, it will be the final vocabulary output.

Which it will be done in [Step 5.2. Create Vocab Output](#output2) 

<div class="alert alert-block alert-warning">
    
### 4.7. Tokens Vectorization<a class="anchor" name="token_vec"></a>
In this task, the aim is to assign each individual tokens with a unique ID.
And eventually using that unique ID to replace each tokens in a token dictionary.

In [18]:
# Create vectorizer_dict from mwe_dict
# vectorizer_dict contains list of potential tokens corresponding to each PID
vectorizer_dict = {}
for key, values in mwe_dict.items():
  token_list = []
  for token in values:
    # While the potential tokens also need to satisfy the requirement that they do not belong to context dependent nor rare token stopwords list
    if token not in context_dependent and token not in rare_tokens:
      token_list.append(token)
  vectorizer_dict[key] = token_list

In [19]:
# vectorizer_dict are derived from mwe_dict which has not been stemmed. 
# Perform stemming on vectorizer_dict:

vectorizer_stemmed = {}
for key, values in vectorizer_dict.items():
  token_list = []
  for token in values:
    if len(stemmer.stem(token)) >= 3:   # length of all tokens have to be larger than 3 after stemming
      token_list.append(stemmer.stem(token))
  vectorizer_stemmed[key] = token_list

In [20]:
# Derive a dictionary from vocab_stemmed defined in task 4.6, which it is a completed list of desired vocab of the PDF file
# the new dict vectorizer_vocab_dict has dictionary keys of each individual words.
# While the corrsponding value for each key is an assigned unique number

vectorizer_vocab_dict = {}
for index, value in enumerate(vocab_stemmed):
  vectorizer_vocab_dict[value] = index

After setting up "vectorizer_vocab_dict" and "vectorizer_stemmed", they could now be merged.

It is merged as a new dictionary "vectorizer_final", with dictionary key remains as the PIDs, while the corresponding values changed. 

Instead of containing all the meaning bigrams/ unigrams themselves, they are now represented by the index assigned to them from our previous step.


In [21]:
vectorizer_vocab_dict
vectorizer_stemmed

{'B00004Z7F0': ['deal',
  'side',
  'nice',
  'comfort',
  'hand',
  'pad',
  'hand',
  'pad',
  'take',
  'look',
  'bought',
  'prefer',
  'style',
  'flat',
  'market',
  'regular',
  'style',
  'the_best',
  'older',
  'model',
  'this_on',
  'usb',
  'cool',
  'addit',
  'the_pric',
  'much_mor',
  'spent',
  'much_mor',
  'bought_thi',
  'the_first',
  'doesn',
  'stop',
  'buy',
  'product',
  'end',
  'product',
  'number',
  'cheap',
  'cheap',
  'reason',
  'didn',
  'didn',
  'this_on',
  'take',
  'care',
  'built',
  'return',
  'dream',
  'extrem',
  'comfort',
  'wast',
  'got_thi',
  'well_worth',
  'the_pric',
  'stop',
  'work',
  'guess',
  'comput',
  'usb',
  'port',
  'usb',
  'work',
  'there_wa',
  'number',
  'light',
  'larger',
  'easier',
  'smaller',
  'run',
  'addit',
  'softwar',
  'requir',
  'fit',
  'releas',
  'driver',
  'number',
  'featur',
  'word',
  'you_want',
  'type',
  'buy',
  'portion',
  'type',
  'quick',
  'data',
  'quickli',
  'annoy

In [22]:
vectorizer_final = {}
# Get keys and values from vectorizer_stemmed
for key, value in vectorizer_stemmed.items():
  vocab_list = []
  for token in value:
  # for each dict[key], its values is now a list of number (index, to be precise), 
  # each number representing a meaningful word
    vocab_list.append(vectorizer_vocab_dict[token])
  vectorizer_final[key] = vocab_list

This could be done because the variable to be looped through: "vectorizer_stemmed" is a dictionary in a format of:
```
vectorizer_stemmed = 
{ 
  [PID1: t1,t2,t3],
  [PID2: t2,t4,t6],....
}
```

While the second variable: vectorizer_vocab_dict is a dictionary in a format of:

```
vectorizer_vocab_dict = 
{
  t1: idx1, t2:idx2, t3:idx3.....
}
```
Such that the merged dictionary: vectorizer_final is in a format like:
```
vectorizer_final =
{
  [PID1: idx(t1), idx(t2), idx(t3)], 
  [PID2: idx(t2), idx(t4), idx(t6)],....
}
```
where "idx(t1)" is referring to the index of the specific token.

<div class="alert alert-block alert-success">
    
## 5. Writing Output Files <a class="anchor" name="write"></a>

Four files need to be generated:
* [Sparse matrix (5.1)](#output1) <br>
* [Vocabulary list (5.2)](#output2) <br>

This is performed in the following sections.

<div class="alert alert-block alert-warning">
    
### 5.1. Count Vectorization and Create Sparse Matrix Output<a class="anchor" name="output1"></a>
After knowing all bigrams/ unigrams exist in every PID's reviews, their frequency of apperance can now be counted under each PID, and return as a Sparese matrix text document by using the CountVectorizer function.

In [23]:
# setup directories
os.chdir("/content/drive/MyDrive")
os.getcwd()

with open("32973381_countVec.txt", "w", encoding = "UTF-8") as f:
  for key, value in vectorizer_final.items():
    cv = CountVectorizer(analyzer = "word")  # define alias for CountVec function
    all_idx = []
    for idx in value:
      all_idx.append(str(idx))
    # Get count of each index appearing under each PID's reviews
    counts = cv.fit_transform([" ".join(all_idx)]).toarray()
    # "fit_transform" method returns the count, while "get_feature_names" method helps retrieving the index for the corresponding count
    feature_names = cv.get_feature_names()
    # Write into "CountVec.txt" line by line in the instructed format: "PID, [index:count]"
    f.write(key + ',')
    for name, count in zip(feature_names, counts[0]): # as counts is an list of length 1, take the first index
      if name == feature_names[-1]:  # no comma for last item of every row
        f.write(f'{name}:{count}')
      else:
        f.write(f'{name}:{count},')
    f.write("\n")

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


<div class="alert alert-block alert-warning">
    
### 5.2. Create Vocab Output<a class="anchor" name="output2"></a>
From the previously saved variable "vocab_stemmed", a vocab output could be created by assigning an index to each tokens like the steps performed in step 4.7
List of vocabulary are written to a file, sorted alphabetically, with their reference codes in front of them. 

In [24]:
# setup directories
os.chdir("/content/drive/MyDrive")
os.getcwd()

# Create vocab output
with open ("32973381_vocab.txt", "w", encoding = "UTF-8") as f:
  for num, values in enumerate(vocab_stemmed):
    f.writelines(f'{values}:{str(num)}')
    f.write('\n') 
  

-------------------------------------

<div class="alert alert-block alert-success">
    
## 6. References <a class="anchor" name="Ref"></a>

[1] NLP in Python-Vectorizing,
https://towardsdatascience.com/nlp-in-python-vectorizing-a2b4fc1a339e, Accessed on 28/08/2022

[2] Explanation on FreqDist function, https://www.educative.io/answers/what-is-freqdist-in-python, Accessed on 28/08/2022

[3] Scikit-learn documentation on feature extracation, https://github.com/scikit-learn/scikit-learn/blob/36958fb24/sklearn/feature_extraction/text.py#L1441, Accessed on 28/08/2022

[4] Youtube video on Sklearn by The Semicolon, https://www.youtube.com/watch?v=RZYjsw6P4nI&ab_channel=TheSemicolon, Accessed on 29/08/2022

[5] Explnation on Python math.ceil() Method, https://www.geeksforgeeks.org/floor-ceil-function-python/, Accessed on 29/08/2022


## --------------------------------------------------------------------------------------------------------------------------